# Convergence speed analysis

This notebook assumes the user is in his own machine.

**Change this if necessary:**

In [20]:
USER = "st76i1_5"
GROUP = "st76i1"
PASSWORD = input('Password: ')

In [21]:
import sys
sys.path.append('/home/viniciustmoraes/Desktop/ST7Project/propagation-code-optimization') # change this

### Generate runs

In [22]:
# make upload
!sshpass -p {PASSWORD} \
    scp -r ../../optimizer/ {USER}@chome.metz.supelec.fr:/usr/users/{GROUP}/{USER}/


In [25]:
# Defines the number of times an algorithm is repeated for statistical analysis
n_stat = int(input('Number of simulations for statistical analysis: '))          

# run algorithm on cluster
runs = [
    "--algorithm hill_climbing --steps 100",
    "--algorithm greedy --steps 10",
    "--algorithm tabu_greedy --steps 10 --hparams '{\"n_tabu\":5}'",
    "--algorithm simulated_annealing --steps 100 --hparams '{\"t0\":20}'",
    "--algorithm csa --steps 100",
    "--algorithm cmaes --steps 100",
]

repeated_runs = []

for i in runs:
    repeated_runs += [i]*n_stat



In [26]:

import pexpect

child = pexpect.spawn(f"sshpass -p {PASSWORD} \
                      ssh -tt {USER}@chome.metz.supelec.fr \
                      srun -p cpu_tp -N 1 -n 32 --pty bash",
                      timeout=5*60)

prompt = f"{USER}@kyle[0-9]+:"

for i in range(len(repeated_runs)):
    occupied = child.expect([prompt, "srun: job [1-9]+ queued and waiting for resources"])

    if occupied == 0:
        cmd = f"python3 -m optimizer.main {repeated_runs[i]} --log convergence_alg{i//n_stat}_run{i%n_stat}.log"
        child.sendline(cmd)
        running = 1
        while running == 1:
            running = child.expect(['Run finished', '\n'])
            output = child.before.decode()
            print(output)
        output = child.after.decode()
        print(output)

    elif occupied == 1:
        raise Exception("Error: no resource available. Try running `mysrun` and `scancel` on the remote machine.")
    
child.close()


 ~st76i1_5@kyle65:~$ python3 -m optimizer.main --algorithm hill_climbing --steps 10 --log convergence_alg0_run0.log
MPI startup(): PMI server not found. Please set I_MPI_PMI_LIBRARY variable if it is not a singleton case.
[info] [Me=0] Args:
	algorithm: hill_climbing
	batch: False
	flexible_shape: False
	hparams: {}
	log: convergence_alg0_run0.log
	phase: deploy
	problem_size: [256, 256, 256]
	program_path: iso3dfd-st7
	seed: 33
	steps: 10
	use_energy: False
[info] [Me=0] Hyperparameters:
[info] [Me=0] real seed: 33
[info] [Me=0] Executed command: /usr/bin/mpirun -np 1 -map-by ppr:1:node:PE=16 python3 -m optimizer.main --algorithm hill_climbing --hparams '{}' --log convergence_alg0_run0.log --phase run --problem_size 256 256 256 --program_path iso3dfd-st7 --seed 33 --steps 10 
Unable to init server: Could not connect: Connection refused
Unable to init server: Impossible de se connecter : Connection refused

(main.py:51759): Gdk-CRITICAL **: 19:11:52.643: gdk_cursor_new_for_display: as

In [27]:
# make get_logs
!mkdir -p tmp/
!sshpass -p {PASSWORD} \
    scp '{USER}@chome.metz.supelec.fr:/usr/users/{GROUP}/{USER}/convergence_alg*.log' ./tmp/

### Convert logs to data

In [ ]:
from optimizer.logger import log_to_list

data = {}
data['hillclimbing'] = log_to_list('tmp/convergence_run_0.log')
data['greedy'] = log_to_list('tmp/convergence_run_1.log')
data['greedytabu'] = log_to_list('tmp/convergence_run_2.log')
data['sa'] = log_to_list('tmp/convergence_run_3.log')
data['csa'] = log_to_list('tmp/convergence_run_4.log')

### Plot graphs

Hill Climbing

In [ ]:
alg = 0

# extract data with flairs (either Initial or New best)
new_data = {'hillclimbing': []}
for line in data['hillclimbing']:
    if 'flair' in line:
       new_data['hillclimbing'].append(line)

for k in range(n_stat):
    #todo


y0 = [float(new_data['hillclimbing'][i]['Cost']) for i in range(0, len(new_data['hillclimbing']))]
x0 = [float(new_data['hillclimbing'][i]['eval']) for i in range(0, len(new_data['hillclimbing']))]

y0raw = [float(data['hillclimbing'][i]['Cost']) for i in range(0, len(data['hillclimbing']))]
x0raw = [float(data['hillclimbing'][i]['eval']) for i in range(0, len(data['hillclimbing']))]

In [ ]:
from matplotlib import pyplot as plt

plt.plot(x0, y0, label='Best solutions path')
plt.plot(x0raw, y0raw, '--', label='Every evaluation path')
plt.legend()
plt.xlabel("No. of evaluations")
plt.ylabel("Cost")
plt.title("Hill Climbing")
plt.grid()

Greedy Hill Climbing

In [ ]:
alg += 1
# no need to only extract data with flairs

y1 = [float(data['greedy'][i]['Cost']) for i in range(0, len(data['greedy']))]
x1 = [float(data['greedy'][i]['eval']) for i in range(0, len(data['greedy']))]

In [ ]:
from matplotlib import pyplot as plt

plt.plot(x1,y1)
plt.xlabel("No. of evaluations")
plt.ylabel("Cost")
plt.title("Greedy Hill Climbing")
plt.grid()

Tabu-Greedy Hill Climbing

In [ ]:
alg += 1
# no need to only extract data with flairs

y2 = [float(data['greedytabu'][i]['Cost']) for i in range(0, len(data['greedytabu']))]
x2 = [float(data['greedytabu'][i]['eval']) for i in range(0, len(data['greedytabu']))]

In [ ]:
from matplotlib import pyplot as plt

plt.plot(x2,y2)
plt.xlabel("No. of evaluations")
plt.ylabel("Cost")
plt.title("Tabu-Greedy Hill Climbing")
plt.grid()

Simulated Annealing

In [ ]:
alg += 1
# extract data with flairs (Initial, New best or Risky choice)
new_data['sa'] = []
for line in data['sa']:
    if 'flair' in line:
       new_data['sa'].append(line)

y3 = [float(new_data['sa'][i]['Cost']) for i in range(0, len(new_data['sa']))]
x3 = [float(new_data['sa'][i]['eval']) for i in range(0, len(new_data['sa']))]

y3raw = [float(data['sa'][i]['Cost']) for i in range(0, len(data['sa']))]
x3raw = [float(data['sa'][i]['eval']) for i in range(0, len(data['sa']))]

In [ ]:
from matplotlib import pyplot as plt

plt.plot(x3, y3, label='Best solutions path')
plt.plot(x3raw, y3raw, '--', label='Every evaluation path')
plt.legend()
plt.xlabel("No. of evaluations")
plt.ylabel("Cost")
plt.title("Simulated Annealing")
plt.grid()

Curious Simulated Annealing (CSA)

In [ ]:
alg += 1
# no need to only extract data with flairs

y4raw = [float(data['csa'][i]['Cost']) for i in range(0, len(data['csa']))]
x4raw = [float(data['csa'][i]['eval']) for i in range(0, len(data['csa']))]

In [ ]:
from matplotlib import pyplot as plt

plt.plot(x4raw, y4raw, '--', label='Every evaluation path')
plt.xlabel("No. of evaluations")
plt.ylabel("Cost")
plt.title("Curious Simulated Annealing")
plt.grid()

### Convergence speed comparison per number of evaluations

In [ ]:
from matplotlib import pyplot as plt

plt.plot(x0, y0, label='Hill Climbing')
plt.plot(x1, y1, label='Greedy Hill Climbing')
plt.plot(x2, y2, label='Tabu-Greedy Hill Climbing')
plt.plot(x3, y3, label='Simulated Annealing')
plt.legend()
plt.xlabel("No. of evaluations")
plt.ylabel("Cost (throughput)")
plt.title("Algorithm convergence speed comparison")
plt.grid()